In [1]:
!pip install python-dotenv

In [2]:
!pip install -U google-generativeai

In [2]:
!pip install -qU langchain-google-genai

In [1]:
import os
from dotenv import load_dotenv
import warnings
from langchain_community.graphs import Neo4jGraph
import fitz
from tqdm.auto import tqdm
import requests
import spacy 
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import textwrap
from langchain_community.vectorstores import Neo4jVector
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
documents = ["My_pdf.pdf",
             "SCI2022-5395248.pdf"
            ]

# path = 'My_pdf.pdf'
# if not os.path.exists(path) :
#     response = request.get('')
#     filename = 'My_pdf.pdf'
#     with open(filename,'wb') as file:
#         file.write(response.content()) 
       

In [3]:
nlp = spacy.load("en_core_web_sm")

def preprocess_text (text : str) -> str:
    txt = text.replace('\n',' ').strip()
    return txt

def transform_pdf_sentences (documents : list())->list(dict()):
    liste = []
    for path in documents:
        doc = fitz.open(path)
        for page_number,page in tqdm(enumerate(doc)):
            sentences = []
            text = preprocess_text(page.get_text())
            spacy_text = nlp(text)
            for sent in spacy_text.sents:
                sentences.append(str(sent))
            liste.append({
                'page_number':page_number,
                'text':str(text),
                'sentences':sentences,
                'tokens_counts':len(text)/4,
                'tokens_per_sentence':(len(text)/4)/len(sentences),
                'pdf_name':path
            })
    return liste

In [4]:
pages_to_dict = transform_pdf_sentences(documents)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [5]:
import pandas as pd

df = pd.DataFrame(pages_to_dict)

df.head()

,page_number,text,sentences,tokens_counts,tokens_per_sentence,pdf_name
0,0,REVIEW Open Access Stem cells: a potential tre...,[REVIEW Open Access Stem cells: a potential tr...,1226.50,39.564516,My_pdf.pdf
1,1,"differentiating, and replacing damaged cells [...","[differentiating, and replacing damaged cells ...",808.00,44.888889,My_pdf.pdf
2,2,cells of any tissue types except for extraembr...,[cells of any tissue types except for extraemb...,1461.00,30.437500,My_pdf.pdf
3,3,"nephron-like structures containing glomeruli, ...","[nephron-like structures containing glomeruli,...",1469.75,32.661111,My_pdf.pdf
4,4,function and structure of the kidney. Injectio...,"[function and structure of the kidney., Inject...",1470.75,33.426136,My_pdf.pdf


In [9]:
df.describe()

In [6]:

num_sentence_chunk_size = 11 # our embedding model accept 384 tokens 

def split_list(input_list: list, slice_size: int) -> list[list[str]]:

    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]


for item in tqdm(pages_to_dict):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],slice_size=num_sentence_chunk_size)
    item["number_of_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/35 [00:00<?, ?it/s]

In [7]:
import re


pages_to_chunks = []
for item in tqdm(pages_to_dict):
    for idx,sentence_chunk in enumerate(item["sentence_chunks"]):
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)  
        chunk_dict["chunk"] = joined_sentence_chunk

        chunk_dict["chunk_pdf_name"] = item['pdf_name']
        chunk_dict["chunk_begin"] = idx*11
        chunk_dict["chunk_end"] = len(sentence_chunk)+idx*11
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_to_chunks.append(chunk_dict)

len(pages_to_chunks)

  0%|          | 0/35 [00:00<?, ?it/s]

193

In [8]:
dff = pd.DataFrame(pages_to_chunks)
dff.head()

,page_number,chunk,chunk_pdf_name,chunk_begin,chunk_end,chunk_token_count
0,0,REVIEW Open Access Stem cells: a potential tre...,My_pdf.pdf,0,11,500.00
1,0,Owing to high medical costs and adverse impact...,My_pdf.pdf,11,22,346.50
2,0,2020Open Access This article is licensed under...,My_pdf.pdf,22,31,376.25
3,1,"differentiating, and replacing damaged cells [...",My_pdf.pdf,0,11,581.25
4,1,"[16, 17]. But understanding TSCs helps us to u...",My_pdf.pdf,11,18,224.75


In [9]:
dff.describe()

,page_number,chunk_begin,chunk_end,chunk_token_count
count,193.000000,193.000000,193.000000,193.000000
mean,11.487047,44.000000,54.062176,245.641192
std,5.794312,44.441792,44.365624,145.954676
min,0.000000,0.000000,7.000000,6.500000
25%,7.000000,11.000000,22.000000,142.000000
50%,13.000000,33.000000,38.000000,201.000000
75%,16.000000,66.000000,77.000000,336.250000
max,19.000000,165.000000,169.000000,848.500000


In [14]:
!pip install neo4j

In [3]:
# from whyhow import WhyHow
load_dotenv()


warnings.filterwarnings("ignore")

#namespace = "ATLAS Graph-RAG"

#client = WhyHow(neo4j_user=user,neo4j_password=password,neo4j_url=url)
kg = Neo4jGraph(
    url='Enter your Neo4j DB URL', username="neo4j", password="Your Password", database='neo4j'
)
# documents_response = client.graph.add_documents(namespace = namespace, documents = documents)


In [11]:
query = """
    MERGE(mergedChunk:Chunk {chunkId: $chunkId})
        ON CREATE SET 
            mergedChunk.text = $chunkParam['chunk'],
            mergedChunk.page = $chunkParam['page_number'], 
            mergedChunk.source = $chunkParam['chunk_pdf_name'], 
            mergedChunk.begin = $chunkParam['chunk_begin'], 
            mergedChunk.end = $chunkParam['chunk_end']
        RETURN mergedChunk
    """      
for idx,chunk in tqdm(enumerate(pages_to_chunks)):
    kg.query(query,params={'chunkParam': chunk,'chunkId':idx})

0it [00:00, ?it/s]

In [12]:
kg.query("""
CREATE CONSTRAINT unique_chunk IF NOT EXISTS 
    FOR (c:Chunk) REQUIRE c.chunkId IS UNIQUE
""")

[]

In [13]:
kg.query("""
         MATCH (n)
         RETURN count(n) as nodeCount
         """)

[{'nodeCount': 193}]

In [14]:
Creating_index_cypher = """
CREATE VECTOR INDEX `vector_index` IF NOT EXISTS
    FOR (c:Chunk) ON c.chunk_embedding
    OPTIONS { indexConfig: {
            `vector.dimensions`: 768,
            `vector.similarity_function`: 'cosine'    
         }}

"""

kg.query(Creating_index_cypher)

[]

In [7]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device=device)

In [16]:
for idx,chunk in tqdm(enumerate(pages_to_chunks)):
    emb = embedding_model.encode(chunk['chunk'])
    kg.query("""
    MATCH (chunk:Chunk) WHERE chunk.chunk_embedding IS NULL AND chunk.chunkId = $ID
    
    WITH chunk, $embedding AS vector
    
    CALL db.create.setNodeVectorProperty(chunk, "chunk_embedding", vector)
    """ ,params={'embedding': emb,'ID':idx})

0it [00:00, ?it/s]

In [8]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Chunk {chunkId: INTEGER, text: STRING, page: INTEGER, source: STRING, begin: INTEGER, end: INTEGER, chunk_embedding: LIST}
PDF {pdf_name: STRING}
Relationship properties:

The relationships:
(:Chunk)-[:IN]->(:PDF)
(:Chunk)-[:NEXT]->(:Chunk)


In [9]:
def neo4j_vector_search(question,embedding_model):
    """Search for similar nodes using the Neo4j vector index"""
    Q_emb = embedding_model.encode(question)
# print(Q_emb)
    vector_search_query = """
        WITH $embedding AS question_embedding
        CALL db.index.vector.queryNodes($index_name, $top_k, question_embedding) yield node, score
        RETURN score, node.text AS text
      """
    similar = kg.query(vector_search_query, 
                     params={
                      'embedding': Q_emb,
                      'index_name':'vector_index', 
                      'top_k': 10})
    return similar

In [10]:
kg.query("""
  SHOW VECTOR INDEXES
  """
)

[{'id': 4,
  'name': 'vector_index',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['chunk_embedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 8, 5, 14, 46, 30, 894000000, tzinfo=<UTC>),
  'readCount': 5}]

In [11]:
result = neo4j_vector_search('what are the Pluripotent stem cells?',embedding_model)
result

[{'score': 0.8417819142341614,
  'text': '[16, 17]. But understanding TSCs helps us to understand the molecular mechanisms behind main- tenance and manipulation of cell fate and is essential to understand how pluripotent cells form in vivo. Pluripotency refers to a stem cell with potential to dif- ferentiate into any of the three germ lineages: endoderm (interior stomach lining, gastrointestinal tract, the lungs), mesoderm (muscle, bone, blood, urogenital), or ecto- derm (epidermal tissues and nervous system)[18]. Pluri- potent stem cells (PSCs) are capable of giving rise to all Fig. 1 Classification of stem cells based on differentiation potential. TSCs, totipotent stem cells; ESCs, embryonic stem cells; iPSCs, induced pluripotent cells; MSCs, mesenchymal stem cells; HSCs, hematopoietic stem cells; NSCs, neural stem cells; RSPCs, renal stem/progenitor cells Liu et al. Stem Cell Research & Therapy     (2020) 11:249 Page 2 of 20'},
 {'score': 0.8155966997146606,
  'text': 'Stem Cell Res

In [21]:
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 4 GB


In [22]:
query = """
       CREATE (p:PDF { pdf_name : $pdf_name})
       RETURN p.pdf_name

"""
for idx,chunk in tqdm(enumerate(documents)):
    kg.query(query,params={'pdf_name': chunk})

0it [00:00, ?it/s]

In [23]:
kg.query("""
       CREATE CONSTRAINT unique_pdf IF NOT EXISTS
       FOR (p:PDF) REQUIRE p.pdf_name IS UNIQUE
""")



[]

In [24]:
kg.query("""
MATCH (c:Chunk),(p:PDF)
WHERE c.source = p.pdf_name
MERGE (c)-[:IN]->(p)
""")

[]

In [10]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Chunk {chunkId: INTEGER, text: STRING, page: INTEGER, source: STRING, begin: INTEGER, end: INTEGER, chunk_embedding: LIST}
PDF {pdf_name: STRING}
Relationship properties:

The relationships:
(:Chunk)-[:IN]->(:PDF)
(:Chunk)-[:NEXT]->(:Chunk)


In [26]:
cypher = """
  MATCH (from_same_pdf:Chunk)
  WHERE from_same_pdf.source = $pdfname
  WITH from_same_pdf
    ORDER BY from_same_pdf.chunkId ASC
  WITH collect(from_same_pdf) as section_chunk_list
    CALL apoc.nodes.link(
        section_chunk_list, 
        "NEXT", 
        {avoidDuplicates: true}
    )
  RETURN size(section_chunk_list)
"""
for pdf in documents:
    kg.query(cypher, params={'pdfname':pdf})

In [57]:

retrieval_query_window = """
MATCH window = (:Chunk)-[:NEXT*0..1]->(node:Chunk)-[:NEXT*0..1]->(:Chunk)
WITH node, window
ORDER BY length(window) DESC LIMIT 1
WITH nodes(window) as chunkList, node
UNWIND chunkList as chunkRows
WITH collect(chunkRows.text) as textList, node
RETURN apoc.text.join(textList, " \n ") as text,
       node.score as score,
       node {.source} AS metadata
"""


In [58]:
kg.query(retrieval_query_window)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: score)} {position: line: 11, column: 13, offset: 310} for query: '\nMATCH window = (:Chunk)-[:NEXT*0..1]->(node:Chunk)-[:NEXT*0..1]->(:Chunk)\nWITH node, window\nORDER BY length(window) DESC LIMIT 1\nWITH nodes(window) as chunkList, node\nUNWIND chunkList as chunkRows\nWITH collect(chunkRows.text) as textList, node\nRETURN apoc.text.join(textList, " \n ") as text,\n       node.score as score,\n       node {.source} AS metadata\n'


[{'text': "REVIEW Open Access Stem cells: a potential treatment option for kidney diseases Dongwei Liu1,2,3,4, Fei Cheng1,2,3,4, Shaokang Pan1,2,3,4 and Zhangsuo Liu1,2,3,4*Abstract The prevalence of kidney diseases is emerging as a public health problem. Stem cells (SCs), currently considered as a promising tool for therapeutic application, have aroused considerable interest and expectations. With self-renewal capabilities and great potential for proliferation and differentiation, stem cell therapy opens new avenues for the development of renal function and structural repair in kidney diseases. Mounting evidence suggests that stem cells exert a therapeutic effect mainly by replacing damaged tissues and paracrine pathways. The benefits of various types of SCs in acute kidney disease and chronic kidney disease have been demonstrated in preclinical studies, and preliminary results of clinical trials present its safety and tolerability. This review will focus on the stem cell-based therap

In [28]:
class MyEmbeddings:
        def __init__(self, model):
            self.model = SentenceTransformer(model, trust_remote_code=True,device=device)
    
        def embed_documents(self, texts: list[str]) -> list[list[float]]:
            return [self.model.encode(t).tolist() for t in texts]
        
        def embed_query(self, text: str) -> list[float]:
            return self.model.encode(text)

In [78]:
import os
from dotenv import load_dotenv 
from langchain.chains import RetrievalQAWithSourcesChain
neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=MyEmbeddings("all-mpnet-base-v2"),
    url='neo4j+s://58c9bf7b.databases.neo4j.io',
    username='neo4j',
    password='ay-_KJMLLoPDmZGyMnsihIqMrJHdN4_eXV51UpNJXt0',
    index_name='vector_index',
    node_label='Chunk',
    text_node_properties=['text'],
    embedding_node_property='chunk_embedding',
    retrieval_query=retrieval_query_window,
)

retriever = neo4j_vector_store.as_retriever()
load_dotenv('keys.env')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if not GOOGLE_API_KEY:
    raise ValueError("Google API key is not set in environment variables.")


genai.configure(api_key=GOOGLE_API_KEY)

model = ChatGoogleGenerativeAI(model="gemini-pro",
                                   client=genai,
                                   temperature=0.3,
                                   )

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=model, 
    chain_type="stuff", 
    retriever=retriever
)

def prettychain(question: str) -> str:
    """Pretty print the chain's response to a question"""
    response = chain({"question": question}, return_only_outputs=True)
    print(response)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2
DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/README.md HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/sentence_bert_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sente

DEBUG:neo4j.pool:[#CDB1]  _: <POOL> picked existing connection bolt-1559
DEBUG:neo4j.pool:[#CDB1]  _: <POOL> checked re_auth auth=None updated=False force=False
DEBUG:neo4j.pool:[#CDB1]  _: <POOL> handing out existing connection
DEBUG:neo4j.io:[#CDB1]  C: TELEMETRY 2  # (<TelemetryAPI.AUTO_COMMIT: 2>)
DEBUG:neo4j.io:[#CDB1]  C: RUN 'CALL dbms.components()' {} {'db': 'neo4j'}
DEBUG:neo4j.io:[#CDB1]  _: <CONNECTION> client state: READY > STREAMING
DEBUG:neo4j.io:[#CDB1]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#CDB1]  S: SUCCESS {}
DEBUG:neo4j.io:[#CDB1]  S: SUCCESS {'t_first': 6, 'fields': ['name', 'versions', 'edition']}
DEBUG:neo4j.io:[#CDB1]  _: <CONNECTION> server state: READY > STREAMING
DEBUG:neo4j.io:[#CDB1]  S: RECORD * 1
DEBUG:neo4j.io:[#CDB1]  S: SUCCESS {'bookmark': 'FB:kcwQFzOWRTl8TWOIQ18zCpU3d8kBmpA=', 'type': 's', 't_last': 0, 'db': 'neo4j'}
DEBUG:neo4j.io:[#CDB1]  _: <CONNECTION> server state: STREAMING > READY
DEBUG:neo4j.pool:[#CDB1]  _: <POOL> released bolt-1559


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=None
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=1299282.515, ttl=0, perf_time=1299283.89 => False
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=neo4j
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=1299283.078, ttl=10, perf_time=1299283.89 => False
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=False, has_server_for_mode=True, table routers={IPv4Address(('58c9bf7b.databases.neo4j.io', 7687))} => True
DEBUG:neo4j.pool:[#0000]  _: <POOL> using existing routing table RoutingTable(database='neo4j' routers={IPv4Address(('58c9bf7b.databases.neo4j.io', 7687))}, readers={IPv4Address(('58c9bf7b.databases.neo4j.io', 7687))}, writers={IPv4Address(('58c9bf7b.databases.neo4j.io', 7687))}, last_updated_time=1299283.078, ttl=10)
DE

In [63]:
# import logging
# logging.basicConfig(level=logging.DEBUG)

# try:
#     retriever = neo4j_vector_store.as_retriever()
#     print("Retriever initialized successfully")
# except Exception as e:
#     logging.error("Error initializing retriever", exc_info=True)

Retriever initialized successfully


In [52]:
# response = model.invoke("tell me about stem cells")
# print(response)

content="**Stem Cells**\n\nStem cells are unspecialized cells that have the potential to develop into any cell type in the body. They are found in both embryos and adults.\n\n**Types of Stem Cells:**\n\n* **Embryonic stem cells (ESCs)**: Derived from the inner cell mass of a blastocyst (early embryo). They are pluripotent, meaning they can develop into any cell type in the body.\n* **Adult stem cells (ASCs)**: Found in various tissues and organs throughout the body. They are multipotent, meaning they can develop into a limited number of cell types specific to their tissue of origin.\n* **Induced pluripotent stem cells (iPSCs)**: Created by reprogramming adult cells back to an embryonic-like state. They are similar to ESCs in their pluripotency.\n\n**Characteristics of Stem Cells:**\n\n* **Self-renewal:** Stem cells can divide and produce new stem cells, maintaining their population over time.\n* **Differentiation:** Stem cells can differentiate into specialized cell types, such as musc

In [79]:

retrieved_context = retriever.get_relevant_documents("tell me about MSCs")

# Inspect or print the retrieved context
for i, context in enumerate(retrieved_context):
    print(f"Context {i+1}: {context}")
print(i+1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=None
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=1299282.515, ttl=0, perf_time=1299301.234 => False
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=neo4j
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=1299283.078, ttl=10, perf_time=1299301.25 => False
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=True, has_server_for_mode=True, table routers={IPv4Address(('58c9bf7b.databases.neo4j.io', 7687))} => False
DEBUG:neo4j.pool:[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('58c9bf7b.databases.neo4j.io', 7687))
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> in: 58c9bf7b.databases.neo4j.io:7687
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> dns resolver out: 34.78.76.49:7687
DEBUG:neo4j.pool:[#0000]  _: <POOL> _acquire rou

DEBUG:neo4j.io:[#CDB1]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#CDB1]  S: SUCCESS {}
DEBUG:neo4j.io:[#CDB1]  S: SUCCESS {'t_first': 4, 'fields': ['text', 'score', 'metadata']}
DEBUG:neo4j.io:[#CDB1]  _: <CONNECTION> server state: READY > STREAMING
DEBUG:neo4j.io:[#CDB1]  S: RECORD * 1
DEBUG:neo4j.io:[#CDB1]  S: SUCCESS {'bookmark': 'FB:kcwQFzOWRTl8TWOIQ18zCpU3d8kBmpA=', 'type': 'r', 't_last': 4, 'db': 'neo4j', 'notifications': [{'severity': 'WARNING', 'description': "One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: score)", 'code': 'Neo.ClientNotification.Statement.UnknownPropertyKeyWarning', 'position': {'column': 13, 'offset': 384, 'line': 11}, 'title': 'The provided property key is not in the database', 'category': 'UNRECOGNIZED'}]}
DEBUG:neo4j.io:[#CDB1]  _: <CONNECTION> server state: STREAMING > READY
DEBUG:neo4j.pool:

Context 1: page_content='indicating engraftment of MSCs [8, 109, 125, 126]. Al- though MSCs home to the injured sites of kidney via blood circulation after transplantation, the renal MSC engraftment was scarce and transient. Most of injected MSCs remain in other blood-rich organs, such as the lung, liver, and spleen. The numbers of MSC-derived epithelial cells appear to be so low to explain tissue re- covery or wound healing (≤0.1%), and the hypothesis could not explain the rapid protective effect of injected MSCs which occurred within 24–48 h as well. All these facts suggest that the effects of MSCs may be mediated by another mechanism. The research conducted by Tögel et al. has made a breakthrough in this field. The study reported that differentiation of rat BM-MSCs into a tubular or endothelial cell phenotype was not observed in an ischemic/reperfusion experimental model of AKI. But the expression of proinflammatory cytokines IL-1β, TNF-α, IFN-γ, and inducible nitric oxide synthase 

In [81]:
prettychain("tell me about HSCs")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=None
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=1299282.515, ttl=0, perf_time=1299351.437 => True
DEBUG:neo4j.pool:[#0000]  _: <POOL> dropping routing table for database=None
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=neo4j
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=1299301.312, ttl=10, perf_time=1299351.453 => True
DEBUG:neo4j.pool:[#0000]  _: <POOL> dropping routing table for database=neo4j
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=True, has_server_for_mode=False, table routers={IPv4Address(('58c9bf7b.databases.neo4j.io', 7687))} => False
DEBUG:neo4j.pool:[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('58c9bf7b.databases.neo4j.io', 7687))
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> in

DEBUG:neo4j.io:[#CDB1]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#CDB1]  S: SUCCESS {}
DEBUG:neo4j.io:[#CDB1]  S: SUCCESS {'t_first': 4, 'fields': ['text', 'score', 'metadata']}
DEBUG:neo4j.io:[#CDB1]  _: <CONNECTION> server state: READY > STREAMING
DEBUG:neo4j.io:[#CDB1]  S: RECORD * 1
DEBUG:neo4j.io:[#CDB1]  S: SUCCESS {'bookmark': 'FB:kcwQFzOWRTl8TWOIQ18zCpU3d8kBmpA=', 'type': 'r', 't_last': 4, 'db': 'neo4j', 'notifications': [{'severity': 'WARNING', 'description': "One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: score)", 'code': 'Neo.ClientNotification.Statement.UnknownPropertyKeyWarning', 'position': {'column': 13, 'offset': 384, 'line': 11}, 'title': 'The provided property key is not in the database', 'category': 'UNRECOGNIZED'}]}
DEBUG:neo4j.io:[#CDB1]  _: <CONNECTION> server state: STREAMING > READY
DEBUG:neo4j.pool:

{'answer': 'HSC stands for hematopoietic stem cells.\n', 'sources': 'SCI2022-5395248.pdf'}
